<a href="https://colab.research.google.com/github/nishJay/taylorGPT/blob/main/taylorGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
db = pd.read_csv('https://raw.githubusercontent.com/nishJay/taylorGPT/main/taylor_swift_lyrics%20-%20Sheet1%20(1).csv?token=GHSAT0AAAAAAB5FS5SCMVCJ3RGEHSUUR27IY7ONQLA')

In [3]:
db = db.rename(columns={'Unnamed: 2': 'Lyrics'})

In [4]:
db.head()

,Album Title,Song Title,Lyrics
0,Taylor Swift (2006),Tim McGraw,He said the way my blue eyes shined\r\nPut tho...
1,Taylor Swift (2006),Picture to Burn,"State the obvious, I didn't get my perfect fan..."
2,Taylor Swift (2006),Teardrops on My Guitar,"Drew looks at me, I fake a smile so he won't s..."
3,Taylor Swift (2006),A Place in This World,"I don't know what I want, so don't ask me\n'Ca..."
4,Taylor Swift (2006),Cold as You,It's junked out joint off a backroad\nA blinki...


In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt



In [6]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from transformers import TFGPT2LMHeadModel

In [8]:
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [9]:
from transformers import  GPT2Tokenizer

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [11]:
db.head()

,Album Title,Song Title,Lyrics
0,Taylor Swift (2006),Tim McGraw,He said the way my blue eyes shined\r\nPut tho...
1,Taylor Swift (2006),Picture to Burn,"State the obvious, I didn't get my perfect fan..."
2,Taylor Swift (2006),Teardrops on My Guitar,"Drew looks at me, I fake a smile so he won't s..."
3,Taylor Swift (2006),A Place in This World,"I don't know what I want, so don't ask me\n'Ca..."
4,Taylor Swift (2006),Cold as You,It's junked out joint off a backroad\nA blinki...


In [12]:
db.describe()

,Album Title,Song Title,Lyrics
count,165,165,165
unique,11,165,165
top,Midnights,Tim McGraw,He said the way my blue eyes shined\r\nPut tho...
freq,23,1,1


In [13]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(db, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(val_data, test_size=0.5, random_state=42)

# Save the training, validation, and test sets to separate CSV files
train_data.to_csv('train.csv', index=False)
val_data.to_csv('val.csv', index=False)
test_data.to_csv('test.csv', index=False)

In [14]:
train_db = pd.read_csv('train.csv')
val_db = pd.read_csv('val.csv')
test_db = pd.read_csv('test.csv')

In [15]:
train_text = '\n'.join(train_db['Lyrics'])
val_text = '\n'.join(val_db['Lyrics'])
test_text = '\n'.join(test_db['Lyrics'])

In [16]:
print(train_text)

We could leave the Christmas lights up 'til January
And this is our place, we make the rules
And there's a dazzling haze, a mysterious way about you dear
Have I known you 20 seconds or 20 years?
Can I go where you go?
Can we always be this close forever and ever?
And ah, take me out, and take me home
You're my, my, my, my
Lover
We could let our friends crash in the living room
This is our place, we make the call
And I'm highly suspicious that everyone who sees you wants you
I've loved you three summers now, honey, but I want 'em all
Can I go where you go?
Can we always be this close forever and ever?
And ah, take me out, and take me home (forever and ever)
You're my, my, my, my
Lover
Ladies and gentlemen, will you please stand?
With every guitar string scar on my hand
I take this magnetic force of a man to be my lover
My heart's been borrowed and yours has been blue
All's well that ends well to end up with you
Swear to be overdramatic and true to my lover
And you'll save all your dirti

In [17]:
print(val_text)

Friends break up, friends get married
Strangers get born, strangers get buried
Trends change, rumors fly through new skies
But I'm right where you left me
Matches burn after the other
Pages turn and stick to each other
Wages earned and lessons learned
But I'm right where you left me

Help, I'm still at the restaurant
Still sitting in a corner I haunt
Cross-legged in the dim light
They say, "What a sad sight", I...
I swear you could hear a hair pin drop
Right when I felt the moment stop
Glass shattered on the white cloth
Everybody moved on, I, I stayed there
Dust collected on my pinned up hair
They expected me to find somewhere
Some perspective, but I sat and stared
Right where you left me

You left me no, you left me no
You left me no choice but to stay here forever
You left me, you left me no, you left me no
You left me no choice but to stay here forever

Did you ever hear about the girl who got frozen?
Time went on for everybody else, she won't know it
She's still twenty-three inside

In [18]:
print(test_text)

Our secret moments in your crowded room
They've got no idea about me and you
There is an indentation in the shape of you
Made your mark on me, a golden tattoo
All of this silence and patience, pining in anticipation
My hands are shaking from holding back from you (ah, ah, ah)
All of this silence and patience, pining and desperately waiting
My hands are shaking from holding back from all this (ha, ha, ha, ha)
Say my name and everything just stops
I don't want you like a best friend
Only bought this dress so you could take it off
Take it off (ha, ha, ha)
Carve your name into my bedpost
'Cause I don't want you like a best friend
Only bought this dress so you could take it off
Take it off (ha, ha, ha, ha)
Inescapable, I'm not even gonna try
And if I get burned, at least we were electrified
I'm spilling wine in the bathtub, you kiss my face and we're both drunk
Everyone thinks that they know us, but they know nothing about
All of this silence and patience, pining in anticipation
My hands ar

In [21]:
sequence_length = 128
batch_size = 16

In [22]:
train_tokens = tokenizer.encode(train_text)
train_dataset = tf.data.Dataset.from_tensor_slices(train_tokens)
train_dataset = train_dataset.padded_batch(batch_size, padded_shapes=(), drop_remainder=True)
print(train_tokens)
print(train_dataset)


Token indices sequence length is longer than the specified maximum sequence length for this model (71423 > 1024). Running this sequence through the model will result in indexing errors


[1135, 714, 2666, 262, 6786, 7588, 510, 705, 47163, 3269, 201, 198, 1870, 428, 318, 674, 1295, 11, 356, 787, 262, 3173, 201, 198, 1870, 612, 338, 257, 41535, 42772, 11, 257, 11428, 835, 546, 345, 13674, 201, 198, 11980, 314, 1900, 345, 1160, 4201, 393, 1160, 812, 30, 201, 198, 6090, 314, 467, 810, 345, 467, 30, 201, 198, 6090, 356, 1464, 307, 428, 1969, 8097, 290, 1683, 30, 201, 198, 1870, 29042, 11, 1011, 502, 503, 11, 290, 1011, 502, 1363, 201, 198, 1639, 821, 616, 11, 616, 11, 616, 11, 616, 201, 198, 43, 2502, 201, 198, 1135, 714, 1309, 674, 2460, 7014, 287, 262, 2877, 2119, 201, 198, 1212, 318, 674, 1295, 11, 356, 787, 262, 869, 201, 198, 1870, 314, 1101, 4047, 13678, 326, 2506, 508, 7224, 345, 3382, 345, 201, 198, 40, 1053, 6151, 345, 1115, 43285, 783, 11, 12498, 11, 475, 314, 765, 705, 368, 477, 201, 198, 6090, 314, 467, 810, 345, 467, 30, 201, 198, 6090, 356, 1464, 307, 428, 1969, 8097, 290, 1683, 30, 201, 198, 1870, 29042, 11, 1011, 502, 503, 11, 290, 1011, 502, 1363, 357, 754,

In [23]:


# Prepare the data for training
def prepare_input_targets(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

train_dataset = train_dataset.batch(sequence_length+1, drop_remainder=True)
train_dataset = train_dataset.map(prepare_input_targets)
train_dataset = train_dataset.shuffle(10000).batch(batch_size, drop_remainder=True)


In [24]:
tf.config.run_functions_eagerly(True)

In [ ]:
epochs = 3
learning_rate = 1e-4

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer])

# Train the model
model.fit(train_dataset, epochs=epochs)

Epoch 1/3
